# Filtros clasicos de visión por computadora parte 2
## Bordes

<div class="alert alert-block alert-success">
<b>Resumen:</b> Este notebook presenta el análisis de algunos filtros clásicos de visión por computadora, como por ejemplo: Canny, Otsu, y la magnitud del gradiente. 
</div>

***

Importación de librerias

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 1. Lectura imagen de prueba

In [ ]:
# Cargar en escala de grises
image = cv2.imread('./images/lena_gray.tif')

# Verificación de que la imagen se haya cargado correctamente
if image is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

# Convertir a RGB para mostrar con Matplotlib
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Revisar dimensiones
print("Shape:", image_gray.shape)

## 2. Otros filtros

In [ ]:
# 1. Filtro Gaussiano para suavizar la imagen (reduce el ruido)
image_gaussian = cv2.GaussianBlur(image_gray, (5, 5), 0)

# 2. Detección de bordes usando Canny
image_canny = cv2.Canny(image, 70, 210)

# 3. Binarización por Otsu
_, image_otsu = cv2.threshold(image_gaussian, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 4. Filtro Laplaciano (detección de bordes por diferencias de segundo orden)
image_laplacian = cv2.Laplacian(image_gaussian, cv2.CV_64F)
image_laplacian = cv2.convertScaleAbs(image_laplacian)  # Escalar al rango 0-255

# 5. Filtro de Sobel (detección de bordes en x e y)
sobelx = cv2.Sobel(image_gaussian, cv2.CV_64F, 1, 0, ksize=5)
sobely = cv2.Sobel(image_gaussian, cv2.CV_64F, 0, 1, ksize=5)
sobel_combined = cv2.convertScaleAbs(sobelx + sobely)  # Combinamos x e y

## 3. Resultados

In [ ]:
# Mostrar las imágenes
images = [image_gray, image_gaussian, image_canny, image_otsu, image_laplacian, sobel_combined]
titles = ['Original', 'Filtro Gaussiano', 'Canny', 'Otsu', 'Laplaciano', 'Sobel']

plt.figure(figsize=(15, 10))
for i in range(len(images)):
    plt.subplot(2, 4, i+1)
    plt.imshow(images[i], cmap='gray')
    plt.title(titles[i])
    plt.axis('off')

plt.tight_layout()
plt.show()

---
<b>¿Qué hace cada filtro?</b>

- <b>Gaussiano:</b> Suaviza la imagen eliminando ruido.
- <b>Canny:</b> Detecta los bordes más nítidos de la imagen.
- <b>Otsu:</b> Binariza la imagen, separando el fondo del primer plano.
- <b>Laplaciano:</b> Detecta bordes a través de la segunda derivada.
- <b>Sobel:</b> Detecta bordes en direcciones específicas (horizontal y vertical).

---
El <b>filtro Canny</b> es un algoritmo de detección de bordes robusto y eficiente, ampliamente utilizado en el procesamiento de imágenes. A alto nivel, este filtro realiza las siguientes operaciones fundamentales:

<b>1. Suavizado de la imagen</b> (Filtro Gaussiano)
- Objetivo: Reducir el ruido en la imagen para evitar que pequeños detalles generen falsos bordes.
- Cómo lo hace: Aplica un filtro Gaussiano, que suaviza la imagen mediante la convolución con una matriz gaussiana. Esto ayuda a suavizar las variaciones rápidas de intensidad.

<b>2. Cálculo del gradiente</b>
- Objetivo: Identificar las regiones de cambio brusco de intensidad en la imagen, que indican la presencia de bordes.
- Cómo lo hace: Usa los operadores de ´Sobel´ para calcular las derivadas en las direcciones horizontal (x) y vertical (y). A partir de estas derivadas, calcula la magnitud del gradiente y su dirección en cada píxel.

<b>3. Supresión no máxima</b>
- Objetivo: Afinar los bordes, eliminando gradientes innecesarios.
- Cómo lo hace: Para cada píxel, verifica si su magnitud es un máximo local en la dirección del gradiente. Si no lo es, el píxel se suprime, lo que afina el borde al mantener solo los puntos más fuertes.

<b>4. Umbral con histéresis</b>
- Objetivo: Determinar los bordes reales, separando los píxeles de bordes fuertes de los débiles.
- Cómo lo hace: Utiliza dos umbrales, uno alto y otro bajo:
  - Los píxeles con gradiente mayor que el umbral alto son considerados bordes fuertes.
  - Los píxeles con gradiente entre el umbral alto y bajo son considerados bordes débiles y se conservan solo si están conectados a bordes fuertes.
   - Los píxeles con gradiente menor que el umbral bajo se eliminan.

<b>Resultado:</b>

Al final del proceso, el filtro Canny produce una imagen binaria en la que los bordes están claramente marcados y refinados, eliminando el ruido y destacando las estructuras importantes de la imagen.

---
El <b>filtro Otsu</b> es un método de binarización automática que se utiliza para separar el primer plano del fondo en una imagen. A un nivel alto de abstracción, realiza las siguientes operaciones:

<b>1. Histograma de intensidades</b>
- Objetivo: Analizar la distribución de los niveles de intensidad de los píxeles de la imagen (en escala de grises).
- Cómo lo hace: Calcula el histograma, que muestra la frecuencia de cada valor de intensidad en la imagen (generalmente de 0 a 255).

<b>2. Selección del umbral óptimo</b>
- Objetivo: Encontrar el mejor umbral de intensidad que divida la imagen en dos grupos: fondo (píxeles oscuros) y primer plano (píxeles brillantes).
- Cómo lo hace: Evalúa todos los posibles umbrales y selecciona el que minimiza la varianza intra-clase y maximiza la varianza inter-clase:
  - Varianza intra-clase: Cuánto varían las intensidades dentro de cada grupo.
  - Varianza inter-clase: Diferencia entre las medias de los dos grupos.
- El umbral óptimo es aquel que logra la máxima separación entre los dos grupos.

<b>3. Aplicación del umbral</b>
- Objetivo: Convertir la imagen de escala de grises a una imagen binaria (blanco y negro).
- Cómo lo hace: Todos los píxeles con un valor de intensidad por debajo del umbral óptimo se asignan a negro (fondo), y los píxeles por encima del umbral se asignan a blanco (primer plano).

<b>Resultado:</b>

El <b>filtro Otsu</b> genera una imagen binarizada donde el umbral ha sido seleccionado de manera automática y óptima, separando eficazmente el fondo y el objeto de interés sin necesidad de intervención manual.